In [17]:
import sys
sys.path.append("..")
import sound_eval as sev
import param_generation as pg
from common_vars import *
import imp
imp.reload(pg)
imp.reload(sev)
import imp
from deap import base, creator
import random
from deap import tools
import scipy.spatial.distance as d
from helpers import rToParams
from IPython.display import Audio


In [148]:
toolbox = base.Toolbox()

class drum_program():
    def __init__(self,stack_num=4):
        self.program=[pg.RandomParams() for i in range(stack_num)]
    def mutate(self):
        for p in self.program:p.mutate(e=0.8,t=0.8,s=0.2)
        return self
creator.create("FitnessMin", base.Fitness, weights=(1,))
creator.create("Individual",drum_program, fitness=creator.FitnessMin)

drum="kick"

def initPopulation(pcls,ng):
    ipop=[creator.Individual()
          for x in range(ng)]
    return pcls(ipop)

def evaluate(individual):
    o=pg.ensemble(individual.program)
    return (sev.evalSoundSolo(o),)

def randSelect(l,k):
    return random.sample(l, k)

# toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list,)


toolbox.register("select_best", tools.selBest,fit_attr='fitness')
toolbox.register("select_random",randSelect)
toolbox.register("evaluate", evaluate)

hof=tools.HallOfFame(10*2)

In [157]:
%%time
GREP MUTPB = 0.5, 1 #making babies,mutation prob,num generations
MUTENV,MUTTEXT=0.3,0.2
POP_SIZE = 8
hof=tools.HallOfFame(POP_SIZE*2)

def evolve(pop,hof):
        newGenes=toolbox.population_guess(int(0.25*POP_SIZE))
        offspring = pop.copy() + newGenes
                        
        # Apply crossover and mutation on the offspring
        for alpha= in hof:
            if random.random() < GREP:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
                
        # Clone the selected individuals        
        for i,mutant in enumerate(offspring):
            if random.random() < MUTPB:
#                 print("before mut",offspring[i])
#                 print(mutant.program)
                offspring[i]=mutant.mutate()
#                 print("after mut ",offspring[i])
                del offspring[i].fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        #update hof
        offspring = toolbox.select_best(offspring,POP_SIZE)
        
        hof.update(offspring)
        return offspring
NGEN=200
def main():
    pop = toolbox.population_guess(POP_SIZE)
#     pop=hof.items
    hof.update(pop)
    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    for g in range(NGEN):
        pop=evolve(pop,hof)
        print (g,"\r",end="")
        for i in hof:
            print(i.fitness)
    return pop
pop=main()
# print([i.fitness.values for i in pop])

(3.5968411307371346e-15,)
(9.589714118022226e-18,)
(2.2180604107328734e-18,)
(6.052515697226537e-19,)
(5.593371520640962e-19,)
(3.471213680495057e-20,)
(1.7647982379349808e-20,)
(1.1702264681868331e-20,)
()
()
()
()
()
()
()
()
(1.775626259359342e-07,)
(3.7534103602965274e-10,)
(5.377365761873032e-13,)
(4.259232753951259e-15,)
(3.5968411307371346e-15,)
(4.046766609352016e-16,)
(9.589714118022226e-18,)
(2.2180604107328734e-18,)
(6.052515697226537e-19,)
(5.593371520640962e-19,)
(5.437911196317746e-20,)
(4.667857744819162e-20,)
(3.471213680495057e-20,)
(1.7647982379349808e-20,)
(1.1702264681868331e-20,)
(2.332471410866418e-21,)
(1.775626259359342e-07,)
(3.620310451424302e-08,)
(3.7534103602965274e-10,)
(8.468553983065341e-12,)
(5.377365761873032e-13,)
(4.259232753951259e-15,)
(3.5968411307371346e-15,)
(4.046766609352016e-16,)
(1.5793586082962797e-17,)
(9.780831717010765e-18,)
(9.589714118022226e-18,)
(2.2180604107328734e-18,)
(9.44586182603012e-19,)
(6.052515697226537e-19,)
(5.59337152064

In [153]:
from ipywidgets import GridspecLayout,HBox,widgets
# What did our DAEP search return?
z=hof.items.copy()
x,y=4,int((len(z)/4))
grid=[]
grid = GridspecLayout(x, y)
for i in range(x):
    for j in range(y):
        o= widgets.Output()
        sample_number=i+j+(i*(y-1))
        pset=z[sample_number]
        try:
            out=pg.ensemble(pset.program)
            with o:
                display(Audio(out,rate=SR,autoplay=False,embed=True),evaluate(z[sample_number]))
            grid[i, j] = o
        except:
            continue
display(grid)

(0.8856837749481201,)

(0.6894107460975647,)

(0.4180154502391815,)

(0.06347418576478958,)

(0.008526205085217953,)

...

(1.7392121520742876e-08,)

(4.541432474525209e-08,)

(5.558304483344045e-09,)

(9.573050130384786e-10,)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [156]:
z=toolbox.population_guess(2)[1]

print(pg.ensemble(z.program))
print(z)
print(evaluate(z))
for x in z.program:
    print(x.__dict__)
    for i in range(10):
        x=x.mutate()
    print(x.__dict__)

Audio(pg.ensemble(z.program),rate=SR,autoplay=True)

[ 0.00000000e+00 -2.02232958e-07 -8.81523512e-07 ...  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
(3.4887198090733926e-22,)
{'oscType': 0, 'isNoise': 0, 'A': 1, 'D': 1, 'S': 2, 'R': 0, 'pitch_0': 2, 'pitch_1': 16, 'pitch_2': 48, 'pitch_3': 80, 'bpCutLow': 72, 'bpCutHigh': 85, 'bpOrder': 2, 'amplitude': 3, 'start': 5, 'length': 4, 'em_prob': 0.5, 'fm_prob': 0.75}
{'oscType': 0, 'isNoise': 0, 'A': 1, 'D': 3, 'S': 0, 'R': 1, 'pitch_0': 10, 'pitch_1': 68, 'pitch_2': 24, 'pitch_3': 0, 'bpCutLow': 12, 'bpCutHigh': 46, 'bpOrder': 0, 'amplitude': 1, 'start': 1, 'length': 0, 'em_prob': 0.5, 'fm_prob': 0.75}
{'oscType': 0, 'isNoise': 1, 'A': 1, 'D': 1, 'S': 3, 'R': 3, 'pitch_0': 6, 'pitch_1': 4, 'pitch_2': 0, 'pitch_3': 16, 'bpCutLow': 50, 'bpCutHigh': 92, 'bpOrder': 2, 'amplitude': 2, 'start': 3, 'length': 4, 'em_prob': 0.25, 'fm_prob': 0.25}
{'oscType': 0, 'isNoise': 1, 'A': 1, 'D': 1, 'S': 3, 'R': 3, 'pitch_0': 94, 'pitch_1': 16, 'pitch_2': 104, 'pitch_3': 0, 'bpCutLow': 53, 'bpCutHigh':